# Genetic Variant Classifications
---

In [2]:
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy import stats
from imblearn.ensemble import EasyEnsemble
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
from sklearn.preprocessing import Imputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
%matplotlib inline

/Users/rook/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Import the dataset

In [126]:
df = pd.read_csv('./data/clinvar_conflicting.csv')
df.shape

/Users/rook/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(65188, 46)

In [127]:
df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,...,SIFT,PolyPhen,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,1,955563,G,C,0.0000,0.00000,0.0000,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,NaN,NaN,NaN,NaN,NaN,NaN,0.421,11.390,1.133255,-2.0
1,1,955597,G,T,0.0000,0.42418,0.2826,MedGen:CN169374,NaN,not_specified,...,NaN,NaN,NaN,NaN,NaN,NaN,0.421,8.150,0.599088,NaN
2,1,955619,G,C,0.0000,0.03475,0.0088,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,NaN,NaN,NaN,NaN,NaN,NaN,0.421,3.288,0.069819,1.0
3,1,957640,C,T,0.0318,0.02016,0.0328,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,NaN,NaN,NaN,NaN,NaN,NaN,0.421,12.560,1.356499,NaN
4,1,976059,C,T,0.0000,0.00022,0.0010,MedGen:CN169374,NaN,not_specified,...,NaN,NaN,NaN,NaN,NaN,NaN,0.421,17.740,2.234711,NaN


### Slightly Imbalanced classes

In [128]:
df['CLASS'].value_counts()

0    48754
1    16434
Name: CLASS, dtype: int64

### Data preperation

In [129]:
X = df.loc[:, ~df.columns.isin(['CLASS'])]
Y = df['CLASS']

In [155]:
# Converting to categorical
convert_cat = []
to_drop = []
unique = None

categorical = X.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    unique = column.nunique()
    print(unique)
    
    # drop cols with too many unique values     
#     if unique <= 100:
    convert_cat.append(i)
#     else:
#         to_drop.append(i)

CHROM
25
REF
866
ALT
458
CLNDISDB
9234
CLNDISDBINCL
48
CLNDN
9260
CLNDNINCL
54
CLNHGVS
65188
CLNSIGINCL
68
CLNVC
7
CLNVI
26289
MC
89
Allele
374
Consequence
48
IMPACT
4
SYMBOL
2328
Feature_type
2
Feature
2369
BIOTYPE
2
EXON
3264
cDNA_position
13970
Protein_position
7339
Amino_acids
1262
Codons
2220
BAM_EDIT
2
SIFT
4
PolyPhen
4
MOTIF_NAME
2
HIGH_INF_POS
1


In [156]:
print(f'convert_cat\n------\n{convert_cat} \n')
# print(f'to_drop\n------\n{to_drop}')

convert_cat
------
['CHROM', 'REF', 'ALT', 'CLNDISDB', 'CLNDISDBINCL', 'CLNDN', 'CLNDNINCL', 'CLNHGVS', 'CLNSIGINCL', 'CLNVC', 'CLNVI', 'MC', 'Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'cDNA_position', 'Protein_position', 'Amino_acids', 'Codons', 'BAM_EDIT', 'SIFT', 'PolyPhen', 'MOTIF_NAME', 'HIGH_INF_POS'] 



### Drop all features with more than 90% NaN's

In [162]:
for c in convert_cat:
    if X[c].isnull().sum() / X.shape[0] > 0.90:
        print(f'{c}: {X[c].isnull().sum() / X.shape[0]}')
        X = X.loc[:, ~X.columns.isin([c])]

CLNDISDBINCL: 0.9988341412529913
CLNDNINCL: 0.9988341412529913
CLNSIGINCL: 0.9988341412529913
MOTIF_NAME: 0.9999693195066577
HIGH_INF_POS: 0.9999693195066577


In [152]:
# Deleting duplicate rows
X = X.loc[:,~X.columns.duplicated()]

# Drop cols with too many unique values
# X = X.loc[:, ~X.columns.isin(to_drop)]

# Get dummies - conver to categroical
# dummy_na=True
# X = pd.get_dummies(data=X, columns=convert_cat)

# CLNDISDBINCL
X = pd.get_dummies(data=X, columns=['CDS_position']) 

# X

# X.loc[X['CHROM'].isnull(), X.columns.str.startswith("CHROM_")] = np.nan
# .fillna(np.nan)

In [122]:
X.loc[df['CLNDISDBINCL'].isnull(), X.columns.str.startswith("CLNDISDBINCL_")] = np.nan

In [150]:
# lol = X.loc[:, X.columns.str.startswith("INTRON_")]

In [ ]:
# lol.columns

In [159]:
X['CLNDISDBINCL'].isnull().sum()

65112

In [144]:
X

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,...,INTRON_96/105,INTRON_96/115,INTRON_96/182,INTRON_96/362,INTRON_97/105,INTRON_97/362,INTRON_98/105,INTRON_98/362,INTRON_99/115,INTRON_99/362
0,1,955563,G,C,0.0000,0.00000,0.0000,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,0,0,0,0,0,0,0,0,0,0
1,1,955597,G,T,0.0000,0.42418,0.2826,MedGen:CN169374,NaN,not_specified,...,0,0,0,0,0,0,0,0,0,0
2,1,955619,G,C,0.0000,0.03475,0.0088,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,0,0,0,0,0,0,0,0,0,0
3,1,957640,C,T,0.0318,0.02016,0.0328,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,0,0,0,0,0,0,0,0,0,0
4,1,976059,C,T,0.0000,0.00022,0.0010,MedGen:CN169374,NaN,not_specified,...,0,0,0,0,0,0,0,0,0,0
5,1,976554,C,G,0.0000,0.01494,0.0256,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,0,0,0,0,0,0,0,0,0,0
6,1,976563,C,T,0.0000,0.00135,0.0098,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,0,0,0,0,0,0,0,0,0,0
7,1,976598,C,T,0.0000,0.00626,0.0056,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,0,0,0,0,0,0,0,0,0,0
8,1,976629,C,T,0.0000,0.01004,0.0411,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,0,0,0,0,0,0,0,0,0,0
9,1,976963,A,G,0.0141,0.00461,0.0126,"MedGen:C3808739,OMIM:615120|MedGen:CN169374",NaN,"Myasthenic_syndrome,_congenital,_8|not_specified",...,0,0,0,0,0,0,0,0,0,0


In [106]:
cols_with_nan = pd.isnull(X).sum() > 0
print(cols_with_nan)

CHROM                                                                                                                                                          False
POS                                                                                                                                                            False
AF_ESP                                                                                                                                                         False
AF_EXAC                                                                                                                                                        False
AF_TGP                                                                                                                                                         False
CLNDNINCL                                                                                                                                                       True
CLNSIGINCL

In [124]:
X.dtypes

CHROM                                                                                                                                                           object
POS                                                                                                                                                              int64
AF_ESP                                                                                                                                                         float64
AF_EXAC                                                                                                                                                        float64
AF_TGP                                                                                                                                                         float64
CLNDNINCL                                                                                                                                                       objec

### Finding all NaN rows

In [109]:
nans = lambda X: X[X.isnull().any(axis=1)]
len(nans(X))

65188

Every row in the dataset has a null attribute.  Imputing data wiil be a necessity here.

### Imputing data

In [108]:
# For numerical data, impute using mean values
numerical_rows = X.select_dtypes(include=['float64', 'int64'])
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(numerical_rows)

# Impute our data
X[numerical_rows.columns] = imp.transform(numerical_rows)

In [110]:
# # For categorical data, impute using mode values
# categorical_rows = X.select_dtypes(include=['uint8'])
# imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
# imp = imp.fit(categorical_rows)

# # Impute the data
# X[categorical_rows.columns] = imp.transform(categorical_rows)

ValueError: Found array with 0 feature(s) (shape=(65188, 0)) while a minimum of 1 is required.

In [111]:
categorical_rows.columns

Index([], dtype='object')

In [22]:
nans = lambda X: X[X.isnull().any(axis=1)]
len(nans(X))

0

Now we have zero rows with NaN values.

### VarianceThreshold

In [153]:
# Removes all low-variance features
def variance_threshold_selector(data, threshold=0.95):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

vt_to_drop = variance_threshold_selector(X)

vt_to_drop.head()

ValueError: could not convert string to float: 'N'

In [28]:
X = X.loc[:, ~X.columns.isin(vt_to_drop)]

### Display correlation Matrix to identify features that need to be dropped

In [29]:
correlation_matrix = X.corr()
display(correlation_matrix)

,AF_ESP,AF_EXAC,AF_TGP,SSR,MOTIF_POS,MOTIF_SCORE_CHANGE,LoFtool,CHROM_1,CHROM_2,CHROM_3,...,SIFT_deleterious_low_confidence,SIFT_tolerated,SIFT_tolerated_low_confidence,PolyPhen_benign,PolyPhen_possibly_damaging,PolyPhen_probably_damaging,PolyPhen_unknown,MOTIF_NAME_Egr1:MA0341.1,MOTIF_NAME_FOXA1:MA0546.1,HIGH_INF_POS_N
AF_ESP,1.000000e+00,8.518704e-01,8.077414e-01,-1.881046e-03,NaN,1.906514e-19,2.632251e-02,2.326319e-02,-2.182190e-02,6.794538e-03,...,-1.142306e-02,-1.743307e-02,3.054199e-03,-1.596152e-02,-4.396333e-02,-7.556392e-02,3.099066e-03,-9.833599e-04,-9.833599e-04,-1.390692e-03
AF_EXAC,8.518704e-01,1.000000e+00,8.056343e-01,5.535895e-03,NaN,7.267337e-19,2.658234e-02,1.657149e-02,-1.881784e-02,7.828141e-03,...,-9.234471e-03,-2.055712e-02,3.787363e-03,-1.742675e-02,-4.422128e-02,-7.266128e-02,6.393419e-03,-9.533059e-04,-9.533059e-04,-1.348189e-03
AF_TGP,8.077414e-01,8.056343e-01,1.000000e+00,-3.899604e-03,NaN,8.206474e-19,2.823596e-02,1.873285e-02,-2.387240e-02,1.592622e-03,...,-1.186737e-02,-1.961782e-02,2.810568e-03,-1.843730e-02,-4.594331e-02,-7.912733e-02,8.101307e-03,-1.004292e-03,-1.004292e-03,-1.420294e-03
SSR,-1.881046e-03,5.535895e-03,-3.899604e-03,1.000000e+00,NaN,-2.955561e-21,5.377127e-03,3.071461e-03,-1.045625e-03,3.400854e-03,...,1.042185e-02,4.826832e-04,-9.273688e-04,-1.302666e-04,4.248090e-03,2.794092e-03,-3.519493e-18,-1.759706e-18,-1.759706e-18,-2.488619e-18
MOTIF_POS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MOTIF_SCORE_CHANGE,1.906514e-19,7.267337e-19,8.206474e-19,-2.955561e-21,NaN,1.000000e+00,-1.735445e-22,5.996419e-19,8.739546e-19,6.586718e-19,...,1.323679e-19,8.627214e-19,4.823063e-19,1.090807e-18,4.243484e-19,9.757325e-19,2.576834e-20,7.071122e-01,-7.071122e-01,-6.762763e-16
LoFtool,2.632251e-02,2.658234e-02,2.823596e-02,5.377127e-03,NaN,-1.735445e-22,1.000000e+00,-4.477337e-02,1.775238e-01,-4.913631e-02,...,-1.860304e-02,-4.081754e-03,-2.103496e-02,-5.330394e-03,-2.844344e-02,-8.215986e-02,2.308825e-03,7.384939e-17,7.384939e-17,1.044396e-16
CHROM_1,2.326319e-02,1.657149e-02,1.873285e-02,3.071461e-03,NaN,5.996419e-19,-4.477337e-02,1.000000e+00,-1.058893e-01,-5.440538e-02,...,-7.826771e-03,-1.781902e-02,2.105153e-03,-1.081050e-02,-8.595165e-03,8.446994e-04,-2.121381e-03,-1.060666e-03,-1.060666e-03,-1.500020e-03
CHROM_2,-2.182190e-02,-1.881784e-02,-2.387240e-02,-1.045625e-03,NaN,8.739546e-19,1.775238e-01,-1.058893e-01,1.000000e+00,-7.855535e-02,...,-2.452850e-02,-1.958588e-02,-2.584160e-02,-3.786557e-02,-2.269305e-02,-4.043020e-02,2.711206e-03,-1.531484e-03,-1.531484e-03,-2.165863e-03
CHROM_3,6.794538e-03,7.828141e-03,1.592622e-03,3.400854e-03,NaN,6.586718e-19,-4.913631e-02,-5.440538e-02,-7.855535e-02,1.000000e+00,...,-1.030979e-02,-3.654833e-03,3.874598e-03,-1.399442e-03,1.483380e-02,-3.521129e-03,-1.573773e-03,-7.868686e-04,-7.868686e-04,-1.112809e-03


In [30]:
# Correlated features to be dropped
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
print(to_drop)

['CHROM_16', 'CHROM_16', 'CLNDNINCL_21-hydroxylase_deficiency', 'CLNDNINCL_3-Oxo-5_alpha-steroid_delta_4-dehydrogenase_deficiency', "CLNDNINCL_Acute_neuronopathic_Gaucher's_disease|Subacute_neuronopathic_Gaucher's_disease|Gaucher_disease,_perinatal_lethal|Gaucher's_disease,_type_1", 'CLNDNINCL_Aortic_aneurysm,_familial_thoracic_4', 'CLNDNINCL_Biotinidase_deficiency', 'CLNDNINCL_Breast-ovarian_cancer,_familial_1', 'CLNDNINCL_Brugada_syndrome_1', "CLNDNINCL_Bull's_eye_maculopathy|Methylmalonic_acidemia_with_homocystinuria", 'CLNDNINCL_Carnitine_palmitoyltransferase_II_deficiency|Carnitine_palmitoyltransferase_II_deficiency,_myopathic,_stress-induced', 'CLNDNINCL_Central_core_disease', 'CLNDNINCL_Charcot-Marie-Tooth_disease', 'CLNDNINCL_Coenzyme_Q10_deficiency,_primary,_4', 'CLNDNINCL_Deafness,_autosomal_dominant_16', 'CLNDNINCL_Deafness,_autosomal_recessive_3', 'CLNDNINCL_Deafness,_autosomal_recessive_7', 'CLNDNINCL_Deafness,_autosomal_recessive_77', 'CLNDNINCL_Deafness,_autosomal_recess

In [31]:
cols = list(X.columns)

for col in to_drop: 
    cols.remove(col)

len(cols)

X = X[cols]

## Train / Test splits

In [32]:
# Train splits
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.40, random_state=42, stratify=Y)

In [33]:
# Dev and Test splits
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)

## Random Forest

In [34]:
rfc = ensemble.RandomForestClassifier(n_estimators=200, class_weight='balanced', n_jobs=-1)

In [35]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

## Evaluation

In [36]:
# Train accuracy
print(f'Train Score: {rfc.score(X_train, y_train)}')
# Dev accuracy
print(f'Dev Score: {rfc.score(X_dev, y_dev)}')

Train Score: 0.9514471262016773
Dev Score: 0.7472771897530296


In [37]:
y_pred_train = rfc.predict_proba(X_train)[:,1]
auc_train = roc_auc_score(y_train, y_pred_train)
print('AUC train', auc_train)

AUC train 0.9849853126320102


In [38]:
y_pred_dev = rfc.predict_proba(X_dev)[:,1]
auc_dev = roc_auc_score(y_dev, y_pred_dev)
print('AUC dev', auc_dev)

AUC dev 0.7483362498341343


In [39]:
dev_pred = rfc.predict(X_dev)
dev_table = pd.crosstab(y_dev, dev_pred, margins=True)
print(pd.crosstab(y_dev, dev_pred))

col_0     0     1
CLASS            
0      8485  1266
1      2029  1258


In [40]:
rf_acc = cross_val_score(rfc, X_train, y_train, cv=5)

In [44]:
print(f'Cross Val: {rf_acc}')
print(f'Average: {np.mean(rf_acc)}')
# Lower bound STD
print('Lower bound STD: ', np.round(rf_acc.mean() - 2 * rf_acc.std(), 2))
# Uppder bound STD
print('Upper bound STD: ', np.round(rf_acc.mean() + 2 * rf_acc.std(), 2))

Cross Val: [0.74306532 0.75265244 0.75031961 0.74558936 0.74405523]
Average: 0.7471363917730673
Lower bound STD:  0.74
Upper bound STD:  0.75


## Feature selection

In [1]:
# estimator = SVR(kernel="linear")
# selector = RFE(estimator, step=1)
# selector = selector.fit(X_train, y_train)

## Gradient Boosting

In [332]:
# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 700,
          'max_depth': 3,
          'loss': 'deviance'}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(X_train, y_train)

predict_train = clf.predict(X_train)
predict_dev = clf.predict(X_dev)

# Accuracy tables.
table_train = pd.crosstab(y_train, predict_train, margins=True)
table_dev = pd.crosstab(y_dev, predict_dev, margins=True)

train_tI_errors = table_train.loc[0.0,1.0] / table_train.loc['All','All']
train_tII_errors = table_train.loc[1.0,0.0] / table_train.loc['All','All']

dev_tI_errors = table_dev.loc[0.0,1.0]/table_dev.loc['All','All']
dev_tII_errors = table_dev.loc[1.0,0.0]/table_dev.loc['All','All']

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(train_tI_errors, train_tII_errors, dev_tI_errors, dev_tII_errors))

Training set accuracy:
Percent Type I errors: 0.041905297606872574
Percent Type II errors: 0.1760840662712211

Test set accuracy:
Percent Type I errors: 0.04701641356036202
Percent Type II errors: 0.18768215984046632


In [333]:
clf.score(X_train, y_train)

0.7820106361219064

In [334]:
clf.score(X_dev, y_dev)

0.7653014265991717

## Logistic Regression

In [48]:
lr = LogisticRegression(C=1e5)

# Fit the model.
fit = lr.fit(X_train, y_train)

pred_y_sklearn = lr.predict(X_dev)

p_sklearn = np.where(pred_y_sklearn < .5, 0, 1)

print('\n Accuracy by admission status')
print(pd.crosstab(y_dev, p_sklearn))

print('\n Percentage accuracy Train')
print(lr.score(X_train, y_train))

print('\n Percentage accuracy')
print(lr.score(X_dev, y_dev))


 Accuracy by admission status
col_0     0   1
CLASS          
0      9714  37
1      3253  34

 Percentage accuracy Train
0.7492329719779096

 Percentage accuracy
0.7476606841540113


## Imbalanced

In [335]:
ee = EasyEnsemble(random_state=0)
X_resampled, y_resampled = ee.fit_sample(X_train, y_train)
print(X_resampled.shape)

rfc_2 = ensemble.RandomForestClassifier(n_estimators=200, class_weight='balanced', max_features='log2', n_jobs=-1)

for x, y in zip(X_resampled, y_resampled):
    rfc_2.fit(x, y)

(10, 19720, 261)


In [336]:
# Train accuracy
print(f'Train Score: {rfc_2.score(X_train, y_train)}')
# Dev accuracy
print(f'Dev Score: {rfc_2.score(X_dev, y_dev)}')

Train Score: 0.8107997545510329
Dev Score: 0.6818530449455438


In [21]:
# X.dtypes

In [20]:
# X[numerical_rows.columns]

In [46]:
# list(X.columns)